## Dynamic Queries

In [27]:
DO $$
DECLARE i INT;
BEGIN
    EXECUTE 'SELECT 42 UNION SELECT 43' INTO STRICT i;
    RAISE NOTICE 'i = %', i;
END $$

---
### Dynamic table names and quote_ident function

In [33]:
DO $$
DECLARE 
    tablename NAME;
    tablenames NAME[] = ARRAY['customers', 'orders'];
    rc BIGINT;
BEGIN
    tablename = tablenames[1 + RANDOM()];
    -- SELECT COUNT(*) from tablename INTO rc;
    EXECUTE 'SELECT COUNT(*) FROM ' || quote_ident(tablename) INTO rc;
    RAISE NOTICE '% has % rows', tablename, rc;
END $$

---
### Quoting literals with quote_literal and quote_nullable

In [40]:
DO $$
DECLARE 
    firstname VARCHAR;
    firstnames VARCHAR[] = ARRAY['A', 'B'];
    tbl_name NAME = 'customers';
    col_name NAME = 'firstname';
    rc BIGINT;
BEGIN
    firstname = firstnames[1+RANDOM()];
    EXECUTE 
        'SELECT COUNT(*) FROM '
        || quote_ident(tbl_name)
        || ' WHERE '
        || quote_ident(col_name) 
        || ' LIKE ' 
        || quote_literal(firstname || '%') INTO rc;

    RAISE NOTICE '% firstnames start with "%"', rc, firstname;
END $$

In [41]:
SELECT 
    lastname, 
    'quoted:' || quote_literal(lastname), 
    'nullable:' || quote_nullable(lastname) 
FROM
    (SELECT 'O''Neil' AS lastname UNION SELECT NULL) s

---
### SQL injection and the USING clause

In [42]:
DROP TABLE IF EXISTS foo;
CREATE TABLE foo (a INT);
DO $$
DECLARE 
    baddata VARCHAR = '0;DROP TABLE IF EXISTS foo;';
BEGIN
    EXECUTE 
        'SELECT count(*) FROM orders where orderid ='
        || baddata;
END $$;
SELECT * from information_schema.tables where table_name = 'foo';   

In [45]:
DO $$
DECLARE 
    firstname VARCHAR;
    firstnames VARCHAR[] = ARRAY['A', 'B'];
    tbl_name NAME = 'customers';
    col_name NAME = 'firstname';
    rc BIGINT;
BEGIN
    firstname = firstnames[1+RANDOM()];
    EXECUTE 
        'SELECT COUNT(*) FROM '
        || quote_ident(tbl_name)
        || ' WHERE '
        || quote_ident(col_name) 
        || ' LIKE $1' 
        INTO rc
        USING firstname || '%';

    RAISE NOTICE '% firstnames start with "%"', rc, firstname;
END $$

In [49]:
DO $$
DECLARE 
    firstname VARCHAR;
    firstnames VARCHAR[] = ARRAY['A', 'B'];
    tbl_name NAME = 'customers';
    col_name NAME = 'firstname';
    rc BIGINT;
BEGIN
    firstname = firstnames[1+RANDOM()];
    EXECUTE 
        format(
            'SELECT COUNT(*) FROM %I WHERE %I LIKE $1', 
            tbl_name, 
            col_name)
        INTO rc
        USING firstname || '%';

    RAISE NOTICE '% firstnames start with "%"', rc, firstname;
END $$

---
### Other uses for dynamic queries
    RETURN QUERY EXECUTE command-string [ USING expression [, ... ] ];
    FOR target IN EXECUTE text_expression [ USING expression [, ... ] ] LOOP...
    OPEN unbound_cursorvar [ [ NO ] SCROLL ] FOR EXECUTE query_string
